# ОИАД. Лабораторная работа №3

**Датасет:** `insurance.csv` (формат: age,sex,bmi,children,smoker,region,charges`)

**Цель работы:** подготовка данных, реализация многомерной линейной регрессии аналитически и численно (градиентный спуск), добавление регуляризации (Ridge), и сравнение моделей по MSE на тесте.

Файл содержит подробные комментарии и блоки кода — вы можете запускать ячейки последовательно в Jupyter Notebook.


In [ ]:
# Блок 0: Импорты и загрузка данных
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Попробуйте заменить путь на ваш local 'insurance.csv' при необходимости
csv_path = 'insurance.csv'
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    # Если файла нет, создадим маленькую демонстрационную таблицу (пользуйтесь своим файлом в реальной работе)
    data = '''age,sex,bmi,children,smoker,region,charges
19,female,27.9,0,yes,southwest,16884.924
18,male,33.77,1,no,southeast,1725.5523
28,male,33,3,no,southeast,4449.462
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552
31,female,25.74,0,no,southeast,3756.6216
46,female,33.44,1,no,southeast,8240.5896
37,female,27.74,3,no,northwest,7281.5056
37,male,29.83,2,no,northeast,6406.4107
60,female,25.84,0,no,northwest,28923.13692
25,male,26.22,0,no,northeast,2721.3208
62,female,26.29,0,yes,southeast,27808.7251
23,male,34.4,0,no,southwest,1826.843
56,female,39.82,0,no,southeast,11090.7178
27,male,42.13,0,yes,southeast,39611.7577
19,male,24.6,1,no,southwest,1837.237
52,female,30.78,1,no,northeast,10797.3362
23,male,23.845,0,no,northeast,2395.17155
56,male,40.3,0,no,southwest,10602.385
30,male,35.3,0,yes,southwest,36837.467
60,female,36.005,0,no,northeast,13228.84695
30,female,32.4,1,no,southwest,4149.736
18,male,34.1,0,no,southeast,1137.011
34,female,31.92,1,yes,northeast,37701.8768
37,male,28.025,2,no,northwest,6203.90175
59,female,27.72,3,no,southeast,14001.1338
'''
    import io
    df = pd.read_csv(io.StringIO(data))

print('Загружено строк:', len(df))
df.head()

In [ ]:
# 1. Подготовка данных
# 1.1 Проверка на пропуски и базовая статистика
print('Пропуски по столбцам:')
print(df.isnull().sum())

print('\nТипы столбцов:')
print(df.dtypes)

print('\nБазовые описательные статистики (числовые признаки):')
print(df.describe())


In [ ]:
# 1.2 Поиск выбросов (IQR и Z-score)
numeric_cols = ['age','bmi','children','charges']

# IQR метод
outliers_iqr = {}
for c in numeric_cols:
    Q1 = df[c].quantile(0.25)
    Q3 = df[c].quantile(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    high = Q3 + 1.5 * IQR
    mask = (df[c] < low) | (df[c] > high)
    outliers_iqr[c] = df[mask]
    print(f"{c}: {mask.sum()} выбросов по IQR (пределы {low:.2f}..{high:.2f})")

# Z-score (для информации)
from scipy import stats
z = np.abs(stats.zscore(df[numeric_cols]))
z_outliers = (z > 3).any(axis=1)
print('\nПо Z-score (>3):', z_outliers.sum(), 'строк')

# Покажем примеры выбросов (если есть)
for c,rows in outliers_iqr.items():
    if len(rows)>0:
        print('\nПримеры выбросов по', c)
        print(rows[[c,'sex','smoker','charges']].head())


In [ ]:
# 1.3 Приведение категориальных признаков к числовым (one-hot и бинарные)
# sex -> binary, smoker -> binary, region -> one-hot

df2 = df.copy()
df2['sex'] = (df2['sex'] == 'male').astype(int)
df2['smoker'] = (df2['smoker'] == 'yes').astype(int)
# region one-hot
df2 = pd.get_dummies(df2, columns=['region'], drop_first=True)

print('Колонки после кодирования:')
print(df2.columns.tolist())

# 1.4 Парные корреляции
corr = df2.corr()
print('\nМатрица корреляций (фрагмент):')
print(corr[['charges']].sort_values(by='charges', ascending=False))

# Покажем heatmap (matplotlib)
plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation='nearest')
plt.title('Корреляционная матрица (все признаки)')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.tight_layout()
plt.show()


In [ ]:
# 2. Многомерная линейная регрессия
# Подготовка X, y
features = [c for c in df2.columns if c != 'charges']
X = df2[features].values.astype(float)
y = df2['charges'].values.reshape(-1,1)

# Добавим столбец единиц для свободного члена
def add_bias(X):
    return np.hstack([np.ones((X.shape[0],1)), X])

# Разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_b = add_bias(X_train)
X_test_b = add_bias(X_test)

# 2.1 Аналитическое решение (обычное МНК)
def normal_equation(X_b, y):
    # w = (X^T X)^{-1} X^T y
    XtX = X_b.T.dot(X_b)
    try:
        inv = np.linalg.inv(XtX)
    except np.linalg.LinAlgError:
        inv = np.linalg.pinv(XtX)
    w = inv.dot(X_b.T).dot(y)
    return w

w_analytical = normal_equation(X_train_b, y_train)
print('Веса (аналитически):')
print(w_analytical.flatten())

# 2.2 Градиентный спуск (реализация)
class LinearGD:
    def __init__(self, lr=0.01, n_iter=10000, tol=1e-6, verbose=False):
        self.lr = lr
        self.n_iter = n_iter
        self.tol = tol
        self.verbose = verbose
    def fit(self, X_b, y):
        n, m = X_b.shape
        self.w = np.zeros((m,1))
        prev_loss = np.inf
        for i in range(self.n_iter):
            preds = X_b.dot(self.w)
            error = preds - y
            grad = (2/n) * X_b.T.dot(error)
            self.w -= self.lr * grad
            loss = np.mean(error**2)
            if self.verbose and i% (self.n_iter//5 if self.n_iter>=5 else 1)==0:
                print(f'iter {i}, loss={loss:.6f}')
            if abs(prev_loss - loss) < self.tol:
                break
            prev_loss = loss
        return self
    def predict(self, X_b):
        return X_b.dot(self.w)

# Для стабильности градиентного спуска масштабируем признаки (не включая bias)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_b_s = add_bias(X_train_scaled)
X_test_b_s = add_bias(X_test_scaled)

gd = LinearGD(lr=0.1, n_iter=20000, tol=1e-8, verbose=False)
_ = gd.fit(X_train_b_s, y_train)
print('Веса (градиентный спуск, на стандартизованных признаках):')
print(gd.w.flatten())


In [ ]:
# 3. Добавление регуляризации (Ridge)
# 3.1 Аналитическое решение для Ridge: w = (X^T X + lambda * I)^-1 X^T y

def ridge_normal_equation(X_b, y, lam):
    m = X_b.shape[1]
    XtX = X_b.T.dot(X_b)
    # не штрафуем свободный член: ставим 0 в верхнем левом элементе
    L = lam * np.eye(m)
    L[0,0] = 0
    try:
        inv = np.linalg.inv(XtX + L)
    except np.linalg.LinAlgError:
        inv = np.linalg.pinv(XtX + L)
    w = inv.dot(X_b.T).dot(y)
    return w

# Подберём lambda по простому сеточному поиску на валидации (train split -> train/val)
X_sub_train, X_val, y_sub_train, y_val = train_test_split(X_train_b, y_train, test_size=0.2, random_state=1)

lambdas = [0.0, 0.01, 0.1, 1, 10, 100]
best_l = None
best_mse = np.inf
for lam in lambdas:
    w_r = ridge_normal_equation(X_sub_train, y_sub_train, lam)
    preds = X_val.dot(w_r)
    mse = mean_squared_error(y_val, preds)
    print(f'lambda={lam}: val MSE={mse:.4f}')
    if mse < best_mse:
        best_mse = mse
        best_l = lam

print('\nЛучший lambda по простому поиску:', best_l)
w_ridge_analytical = ridge_normal_equation(X_train_b, y_train, best_l)
print('Веса Ridge (аналитически):')
print(w_ridge_analytical.flatten())

# 3.2 Градиентный спуск с регуляризацией (L2)
class RidgeGD(LinearGD):
    def __init__(self, lr=0.01, n_iter=10000, lam=1.0, tol=1e-6, verbose=False):
        super().__init__(lr=lr, n_iter=n_iter, tol=tol, verbose=verbose)
        self.lam = lam
    def fit(self, X_b, y):
        n, m = X_b.shape
        self.w = np.zeros((m,1))
        prev_loss = np.inf
        for i in range(self.n_iter):
            preds = X_b.dot(self.w)
            error = preds - y
            grad = (2/n) * (X_b.T.dot(error) + self.lam * np.vstack([ [0.0], self.w[1:]] ))
            self.w -= self.lr * grad
            loss = np.mean(error**2) + self.lam * (self.w[1:]**2).sum()
            if self.verbose and i% (self.n_iter//5 if self.n_iter>=5 else 1)==0:
                print(f'iter {i}, loss={loss:.6f}')
            if abs(prev_loss - loss) < self.tol:
                break
            prev_loss = loss
        return self

# Обучим RidgeGD на стандартизованных данных
rgd = RidgeGD(lr=0.05, n_iter=20000, lam=best_l, tol=1e-8, verbose=False)
rgd.fit(X_train_b_s, y_train)
print('Веса Ridge GD (стандартизованные признаки):')
print(rgd.w.flatten())


In [ ]:
# 4. Оценка обобщающей способности
# 4.1 Бейзлайн: константный предсказатель (среднее по train)
mean_pred = np.mean(y_train)
mse_baseline = mean_squared_error(y_test, np.full_like(y_test, mean_pred))
print('Baseline (mean) MSE:', mse_baseline)

# 4.2 Аналитическая линейная модель
pred_analytical = X_test_b.dot(w_analytical)
mse_analytical = mean_squared_error(y_test, pred_analytical)
print('Analytical OLS MSE:', mse_analytical)

# 4.3 GD (на стандартизованных)
pred_gd = X_test_b_s.dot(gd.w)
mse_gd = mean_squared_error(y_test, pred_gd)
print('Gradient Descent MSE:', mse_gd)

# 4.4 Ridge analytical
pred_ridge_a = X_test_b.dot(w_ridge_analytical)
mse_ridge_a = mean_squared_error(y_test, pred_ridge_a)
print('Ridge Analytic MSE (lambda={}):'.format(best_l), mse_ridge_a)

# 4.5 Ridge GD (стандартизованные)
pred_ridge_gd = X_test_b_s.dot(rgd.w)
mse_ridge_gd = mean_squared_error(y_test, pred_ridge_gd)
print('Ridge GD MSE:', mse_ridge_gd)

# Сводная таблица
results = pd.DataFrame({
    'model': ['baseline_mean','ols_analytic','gd','ridge_analytic','ridge_gd'],
    'mse': [mse_baseline, mse_analytical, mse_gd, mse_ridge_a, mse_ridge_gd]
}).sort_values('mse')

print('\nСравнение моделей (по MSE на тесте):')
print(results)


## Выводы

- В ноутбуке реализованы: проверка данных, кодирование категориальных признаков, проверка выбросов (IQR, Z-score), корреляции, аналитическое решение линейной регрессии (нормальное уравнение), градиентный спуск, добавление L2-регуляризации (Ridge) аналитически и численно, и сравнение моделей по MSE.

- В реальной работе рекомендуется: масштабировать признаки перед обучением градиентных методов, проводить кросс-валидацию для выбора параметра регуляризации, визуализировать остатки и проводить диагностику многоколлинеарности (VIF).


In [ ]:
# Сохраним итоговый ноутбук и веса
import numpy as _np
weights = {
    'w_ols': w_analytical.flatten(),
    'w_ridge_analytic': w_ridge_analytical.flatten(),
}
_np.savez('lab3_weights.npz', **weights)

import nbformat as _nbf
out_path = '/mnt/data/insurance_lab3.ipynb'
with open(out_path, 'w', encoding='utf-8') as f:
    _nbf.write(nb, f)

print('Сохранён ноутбук:', out_path)
